In [1]:
import pandas as pd
import numpy as np

In [2]:
numeric = "../input_orig/train_numeric.csv"
date = "../input_orig/train_date.csv"
categorical = "../input_orig/train_categorical.csv"

numeric_te = "../input_orig/test_numeric.csv"
date_te = "../input_orig/test_date.csv"
categorical_te = "../input_orig/test_categorical.csv"

In [3]:
def orgainize(features):
    line_features = {}
    station_features = {}
    lines = set([f.split('_')[0] for f in features])
    stations = set([f.split('_')[1] for f in features])
    
    for l in lines:
        line_features[l] = [f for f in features if l+'_' in f]
        
    for s in stations:
        station_features[s] = [f for f in features if s+'_' in f]
            
    return line_features, station_features

In [4]:
features_num = pd.read_csv(numeric, nrows=1).drop(['Response', 'Id'], axis=1).columns.values
line_features_num, station_features_num = orgainize(features_num)
print("Features in Station 32: {}".format( station_features_num['S32'] ))

Features in Station 32: ['L3_S32_F3850']


In [5]:
features_date = pd.read_csv(date, nrows=1).drop(['Id'], axis=1).columns.values
line_features_date, station_features_date = orgainize(features_date)
print("Features in Station 32: {}".format( station_features_date['S32'] ))

Features in Station 32: ['L3_S32_D3852']


In [7]:
features_cat = pd.read_csv(categorical, nrows=1).drop(['Id'], axis=1).columns.values
line_features_cat, station_features_cat = orgainize(features_cat)
print("Features in Station 32: {}".format( station_features_cat['S32'] ))

Features in Station 32: ['L3_S32_F3851', 'L3_S32_F3853', 'L3_S32_F3854']


In [8]:
CHUNKSIZE = 200000
NROWS_TR = 1183747
nrows_tr = 0
NROWS_TE = 1183748
nrows_te = 0
ID_COLUMN = 'Id'
TARGET_COLUMN = 'Response'
SEED = 0

tr_minmaxdiff_full_date = pd.DataFrame()
for tr in pd.read_csv(date, chunksize=CHUNKSIZE, dtype=np.float32):
    tmp1 = pd.DataFrame()
    feats = np.setdiff1d(tr.columns, [ID_COLUMN])
    tmp1['date_max_full'] = tr[feats].max(axis=1)
    tmp1['date_min_full'] = tr[feats].min(axis=1)
    tmp1['date_diff_full'] = tmp1['date_max_full'] - tmp1['date_min_full']
    tr_minmaxdiff_full_date = tr_minmaxdiff_full_date.append(tmp1)

    nrows_tr += CHUNKSIZE
    if nrows_tr >= NROWS_TR:
        break
        
te_minmaxdiff_full_date = pd.DataFrame()
for te in pd.read_csv(date_te, chunksize=CHUNKSIZE, dtype=np.float32):
    tmp2 = pd.DataFrame()
    feats = np.setdiff1d(te.columns, [ID_COLUMN])
    tmp2['date_max_full'] = te[feats].max(axis=1)
    tmp2['date_min_full'] = te[feats].min(axis=1)
    tmp2['date_diff_full'] = tmp2['date_max_full'] - tmp2['date_min_full']
    te_minmaxdiff_full_date = te_minmaxdiff_full_date.append(tmp2)
    
    nrows_te += CHUNKSIZE
    if nrows_te >= NROWS_TE:
        break

In [9]:
tr_minmaxdiff_full_date.shape, te_minmaxdiff_full_date.shape

((1183747, 3), (1183748, 3))

In [10]:
te_minmaxdiff_full_date.head()

,date_max_full,date_min_full,date_diff_full
0,800.700012,792.719971,7.980042
1,1060.069946,1025.630005,34.439941
2,711.080017,671.919983,39.160034
3,256.279999,255.449997,0.830002
4,770.280029,743.380005,26.900024


In [11]:
tr_minmaxdiff_full_date.head()

,date_max_full,date_min_full,date_diff_full
0,87.290001,82.239998,5.050003
1,1315.750000,1313.119995,2.630005
2,1624.420044,1618.699951,5.720093
3,1154.160034,1149.199951,4.960083
4,606.020020,602.640015,3.380005


In [12]:
tr_minmaxdiff_full_date.to_hdf('feats/tr_minmaxdiff_full_date9.hdf5','table')
te_minmaxdiff_full_date.to_hdf('feats/te_minmaxdiff_full_date9.hdf5','table')